In [336]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [335]:
# seasons = ['f24', 's24', 'f23', 's23', 'f22','s22']
# seasons = ['f24', 's24', 'f23', 's23', 'f22','s22']
seasons = [[f"f{i}",f"s{i}"] for i in range (16,25)]
seasons = [sub for s in seasons for sub in s]
# seasons = ['s24']

# df_races = pd.DataFrame()
# try:
#   df_races = pd.read_csv("racesf24s24f23.csv")
# except:
df_races = pd.DataFrame(columns=["Score", "Sailor","Link", "GradYear", "Position", "Partner", "Venue", "Regatta", "Scoring", "raceID", "Date", "Week", "Team", "Teamlink"])

regattas = {}
for season in seasons:
  url = f"https://scores.collegesailing.org/{season}/"
  page = requests.get(url)
  listSoup = BeautifulSoup(page.content, 'html.parser')
  
  tbody = listSoup.find('table', class_="season-summary").find('tbody')
  
  for link in tbody.find_all("a", href=True):
    if (season + "/" + link['href']) not in df_races['Regatta'].unique():
      scoring = link.parent.next_sibling.next_sibling.next_sibling.text
      if (scoring == "Team"):
        regattas[season + "/" + link['href']] = {"link":season + "/" + link['href'], "scoring":scoring}

In [342]:
regattaSoups = {}

for i, regatta in enumerate(list(regattas.values())):
    link = regatta['link']
    
    if not os.path.exists(f"pagesTR/{link.split("/")[0]}"):
        os.makedirs(f"pagesTR/{link.split("/")[0]}")
    if os.path.exists(f"pagesTR/{link}-allraces.html") and os.path.exists(f"pagesTR/{link}-sailors.html") and os.path.exists(f"pagesTR/{link}-report.html"):
        with open(f"pagesTR/{link}-allraces.html", "r") as f:
            allRaces = BeautifulSoup(f.read(), 'html.parser')
        with open(f"pagesTR/{link}-sailors.html", "r") as f:
            sailors = BeautifulSoup(f.read(), 'html.parser')
        with open(f"pagesTR/{link}-report.html", "r") as f:
            report = BeautifulSoup(f.read(), 'html.parser')
    else:
        betterVenue = list(regattas.keys())[i]
        print(f"({i + 1}/{len(list(regattas.values()))}) getting soup for {betterVenue}")
        # all races
        url = f"https://scores.collegesailing.org/{regatta['link']}/all/"
        page = requests.get(url)
        with open(f"pagesTR/{link}-allraces.html", "w+") as f:
                        f.write(str(page.content))
        allRaces = BeautifulSoup(page.content, 'html.parser')

        # sailors
        url = f"https://scores.collegesailing.org/{regatta['link']}/sailors/"
        page = requests.get(url)
        with open(f"pagesTR/{link}-sailors.html", "w+") as f:
                        f.write(str(page.content))
        sailors = BeautifulSoup(page.content, 'html.parser')
        
        url = f"https://scores.collegesailing.org/{regatta['link']}/"
        page = requests.get(url)
        with open(f"pagesTR/{link}-report.html", "w+") as f:
                        f.write(str(page.content))
        report = BeautifulSoup(page.content, 'html.parser')
    
    regattaSoups[betterVenue] = {"allRaces": allRaces, "sailors": sailors, 'report': report, "scoring": regatta['scoring']}

(1/452) getting soup for f16/hap-moore-team-race-backup
(2/452) getting soup for f16/franklin-lane-backup
(3/452) getting soup for f16/streufert
(4/452) getting soup for f16/2016-laserperformance-collegiate-cup
(5/452) getting soup for s16/laserperformance-team-race-nationals
(6/452) getting soup for s16/nwicsa-team-race
(7/452) getting soup for s16/seisa-team-racing-champs
(8/452) getting soup for s16/pccsc-team-race-carter-ford
(9/452) getting soup for s16/mcsa-team-race
(10/452) getting soup for s16/new-england-team-race-fowle
(11/452) getting soup for s16/capt-prosser
(12/452) getting soup for s16/big-10-team-race
(13/452) getting soup for s16/mystic-lake-team-race
(14/452) getting soup for s16/lynne-marchiando
(15/452) getting soup for s16/camel-team-race
(16/452) getting soup for s16/herring-pond-team-race
(17/452) getting soup for s16/oxford-classicteam-race
(18/452) getting soup for s16/maisa-team-race-challenge
(19/452) getting soup for s16/aaron-szambecki-team-race
(20/452) g

In [343]:
def makeRaceList(raceRows):
    raceList = []
    round = ''
    
    # loop through races
    for raceRow in raceRows: 
        if raceRow.has_attr('class') and raceRow.attrs['class'][0] == 'roundrow':
            round = raceRow.text
            continue
        
        raceData = [el.text for el in raceRow.find_all('td')]
        raceNum = raceData[0]
        
        teamAEl = raceRow.find_all('td', class_='team1')
        if(len(teamAEl) < 3):
            print("No score for this race?", raceNum)
            continue
        teamAName = list(teamAEl[1].stripped_strings)[0]
        teamANick = list(teamAEl[1].stripped_strings)[1]
        teamALink = teamAEl[1].find('a').attrs['href']
        teamAScore = teamAEl[2].text
        teamAWin = teamAEl[1].attrs['class'][0].split("-")[1]
        teamA = {'name': teamAName, 'nick': teamANick, 'link': teamALink.split("/")[2], 'outcome': teamAWin, 'score': teamAScore}
            
        teamBEl = raceRow.find_all('td', class_='team2')
        teamBName = list(teamBEl[1].stripped_strings)[0]
        teamBNick = list(teamBEl[1].stripped_strings)[1]
        teamBLink = teamBEl[1].find('a').attrs['href']
        teamBScore = teamBEl[0].text
        teamBWin = teamBEl[1].attrs['class'][0].split("-")[1]
        teamB = {'name': teamBName, 'nick': teamBNick, 'link': teamBLink.split("/")[2], 'outcome': teamBWin, 'score': teamBScore}
        # print(raceNum, winningTeam, winningScore, losingTeam, losingScore)
        
        raceList.append({'raceNum': int(raceNum),
                         'round': round,
                         'teamA': teamA,
                         'teamB': teamB,
                         })
    return raceList

In [391]:
def getSailorLinks(reportPage):
    sailorLinks = []
    
    mainTable = reportPage.find('table', class_='results').contents[1]
    allSailors = mainTable.find_all('td', class_='sailor')
    
    for sailor in allSailors:
        sailorName = sailor.text.split(" '")[0].strip()
        sailorLink = 'Unknown'
        if sailor.find('a') != None:
            sailorLink = sailor.find('a').attrs['href'].split("/")[2]
        if sailorName not in [p['name'] for p in sailorLinks]:
            sailorLinks.append({'name': sailorName, 'link':sailorLink})
    return sailorLinks

def getTeamReportInfo(reportPage):
    teamInfoList = []
    
    mainTable = reportPage.find('table', class_='results').contents[1]
    teamRows = mainTable.find_all('tr', class_='topborder')
    
    for row in teamRows:
        teamRank = row.contents[1].text
        teamID = row.attrs['class'][2].split("-")[1]
        teamNick = row.find('td', class_='teamname').text
        uniName = row.find('a').text
        teamLink = row.find('a').attrs['href']
        tiebreaker = {'char': row.find('td', class_='tiebreaker').text,'text':row.find('td', class_='tiebreaker').attrs['title']}
        
        teamInfoList.append({'rank': teamRank, 'uniName': uniName, 'teamNick': teamNick, 'teamID': teamID, 'teamLink': teamLink, 'tiebreaker': tiebreaker})
    return teamInfoList

In [392]:
def makeSailorList(sailorData, regatta):
    sailorRaceList = []
    
    ports = sailorData.find_all('div', class_='port')
    
    round = ''
    for port in ports:
        round = port.find('h3').text
        numTables = len(port.find_all('table'))
        if numTables == 0:
            return [{'name': 'no sailors!'}]
        tableRows = port.find('table').contents[0].contents
    
        opponentsRow = tableRows[0].find_all('th')
        teamNames = [team.text for team in opponentsRow]
        
        fullTeamNames = [el.find_all('th')[0].text for el in tableRows[1:]]
        teamIDs = [el.attrs['class'][1].split("-")[1] for el in tableRows[1:]]
        
        for tableRow, teamName, fullTeamName, teamID in zip(tableRows[1:], teamNames, fullTeamNames, teamIDs):
            sailorTables = tableRow.find_all("table", class_='tr-boats')
            tempNames = teamNames.copy()
            tempNames.remove(teamName)
            tempFullNames = fullTeamNames.copy()
            tempFullNames.remove(fullTeamName)
            
            tempTeamIDs = teamIDs.copy()
            tempTeamIDs.remove(teamID)
            
            for race, opponent, opponentFull, oppID in zip(sailorTables, tempNames, tempFullNames, tempTeamIDs):
                boats = [list(names.stripped_strings) for names in race.find_all('td')]
                for boat in boats:
                    skipperName, skipperYear, crewName, crewYear = ['Unknown'] * 4
                    
                    if len(boat) == 2:
                        if boat[0] != 'No show':
                            skipperName = boat[0].split(" '")[0].strip()
                            skipperYear = boat[0].split(" '")[1].strip()
                        if boat[1] != 'No show':
                            crewName = boat[1].split(" '")[0].strip()
                            crewYear = boat[1].split(" '")[1].strip()
                        
                    sailorRaceList.append({'name': skipperName,
                                           'year': skipperYear,
                                           'pos': 'skipper',
                                           'round': round,
                                           'teamName': teamName, 
                                           'teamID': teamID,
                                           'fullName': fullTeamName, 
                                           'opponent': opponent,
                                           'opponentFull': opponentFull,
                                           'oppID': oppID,
                                           'partner': crewName,
                                           'regatta': regatta
                                           })
                    
                    sailorRaceList.append({'name': crewName,
                                           'year': crewYear,
                                           'pos': 'crew',
                                           'round': round,
                                           'teamName': teamName, 
                                           'teamID': teamID,
                                           'fullName': fullTeamName, 
                                           'opponent': opponent,
                                           'opponentFull': opponentFull,
                                           'oppID': oppID,
                                           'partner': skipperName,
                                           'regatta': regatta
                                           })
                    
    return sailorRaceList

def getTeamIDs(sailorData):
    data = []
    ports = sailorData.find_all('div', class_='port')
    
    for port in ports:
        tableRows = port.find('table').contents[0].contents
        opponentsRow = tableRows[0].find_all('th')
        
        for team, el in zip(opponentsRow, tableRows[1:]):
            teamName = team.text
            uniName = el.find('th').text
            teamID = el.attrs['class'][1].split("-")[1]
            if teamID not in [d['teamID'] for d in data]:
                data.append({'teamName': teamName, 'uniName': uniName, 'teamID': teamID})
    
    return data

In [389]:
data = []
totalSailors = {}
df_totalSailors = pd.DataFrame()

for i, regatta in enumerate(list(regattaSoups.keys())):
# regatta = list(regattaSoups.keys())[0]
# regatta = 's24/41st-cardinal-red-lynne-marchiando'
    print(f"({i + 1}/{len(list(regattas.values()))}) analyzing {regatta}")

    allRaces = regattaSoups[regatta]['allRaces']
    sailors = regattaSoups[regatta]['sailors']
    reportPage = regattaSoups[regatta]['report']
    scoring = regattaSoups[regatta]['scoring']

    if len(allRaces.find_all('table', class_="teamscorelist")) == 0: 
        print(f"no scores entered for {regatta}, skipping")
        continue

    scoreData = allRaces.find_all('table', class_="teamscorelist")[
        0].contents

    host = allRaces.find("span", itemprop='location').text
    date = allRaces.find("time").attrs['datetime']
    date = date[:10]

    regattaType = allRaces.find("span", itemprop='description').text

    raceRows = [row for table in scoreData for row in table.contents][1:]

    df_races = pd.DataFrame(makeRaceList(raceRows))

    df_sailors = pd.DataFrame(makeSailorList(sailors, regatta))
    if len(df_sailors['name'].unique()) < 2:
        print("No rp ented for this regatta", regatta)
        continue

    df_sailorLinks = pd.DataFrame(getSailorLinks(reportPage))

    df_teamReportInfo = pd.DataFrame(getTeamReportInfo(reportPage))
    df_sailorteamInfo = pd.DataFrame(getTeamIDs(sailors))

    # merge dfs 
    df_sailors = pd.merge(df_sailors, df_sailorLinks, how='left', on='name')
    # df_totalSailors = pd.concat([df_totalSailors, df_sailors])

    for _, race_result in df_races.iterrows():
        raceNum = race_result['raceNum']
        round = race_result['round']
        
        teamA = race_result['teamA']
        teamAID = df_teamReportInfo.loc[df_teamReportInfo['uniName'] == teamA['name'], 'teamID'].iat[0]
        # print(teamAID,df_sailorteamInfo)
        teamAName = df_sailorteamInfo.loc[df_sailorteamInfo['teamID'] == teamAID, 'teamName'].iat[0]
        
        teamB = race_result['teamB']
        teamBID = df_teamReportInfo.loc[df_teamReportInfo['uniName'] == teamB['name'], 'teamID'].iat[0]
        teamBName = df_sailorteamInfo.loc[df_sailorteamInfo['teamID'] == teamBID, 'teamName'].iat[0]

        teamASailors = df_sailors.loc[(df_sailors['round'] == round) & (df_sailors['teamID'] == teamAID) & (df_sailors['oppID'] == teamBID)]
        teamABoats = []
        if len(teamASailors) > 0:
            skippers = teamASailors.loc[teamASailors['pos'] == 'skipper']
            crews = teamASailors.loc[teamASailors['pos'] == 'crew']
            for _, skipper in skippers.iterrows():
                crew = crews.loc[crews['partner'] == skipper['name']].iloc[0]
                
                skipperKey = skipper['link'] if skipper['link'] != 'Unknown' else skipper['name'] + "-" + teamAName
                crewKey = crew['link'] if crew['link'] != 'Unknown' else crew['name'] + "-" + teamAName
                
                totalSailors[skipperKey] = {'name': skipper['name'], 'year': skipper['year'], 'link': skipper['link'], 'key': skipperKey}
                totalSailors[crewKey] = {'name': crew['name'], 'year': crew['year'], 'link': crew['link'], 'key': crewKey}
                
                teamABoats.append({'skipperName': skipper['name'], 'skipperLink': skipper['link'],'skipperKey': skipperKey, 
                                    'crewName': crew['name'], 'crewLink': crew['link'], 'crewKey': crewKey})
        
        teamBSailors = df_sailors.loc[(df_sailors['round'] == round) & (df_sailors['teamID'] == teamBID) & (df_sailors['oppID'] == teamAID)]
        teamBBoats = []

        if len(teamBSailors) > 0:
            skippers = teamBSailors.loc[teamBSailors['pos'] == 'skipper']
            crews = teamBSailors.loc[teamBSailors['pos'] == 'crew']
            for _, skipper in skippers.iterrows():
                crew = crews.loc[crews['partner'] == skipper['name']].iloc[0]
                
                skipperKey = skipper['link'] if skipper['link'] != 'Unknown' else skipper['name'] + "-" + teamBName
                crewKey = crew['link'] if crew['link'] != 'Unknown' else crew['name'] + "-" + teamBName
                
                totalSailors[skipperKey] = {'name': skipper['name'], 'year': skipper['year'], 'link': skipper['link'], 'key': skipperKey}
                totalSailors[crewKey] = {'name': crew['name'], 'year': crew['year'], 'link': crew['link'], 'key': crewKey}
                
                teamBBoats.append({'skipperName': skipper['name'], 'skipperLink': skipper['link'],'skipperKey': skipperKey, 
                                    'crewName': crew['name'], 'crewLink': crew['link'], 'crewKey': crewKey})
        
        data.append({'raceID': f"{regatta}/{raceNum}",
                    'raceNum': raceNum, 'round': round,
                    'date': date,
                    'teamAName': teamAName,
                    'teamAUni': teamA['name'],
                    'teamANick': teamA['nick'],
                    'teamALink': teamA['link'],
                    'teamAID': teamAID,
                    'teamABoats': teamABoats,
                    'teamAScore': teamA['score'],
                    'teamAOutcome': teamA['outcome'],
                    'teamBName': teamBName,
                    'teamBUni': teamB['name'],
                    'teamBNick': teamB['nick'],
                    'teamBLink': teamB['link'],
                    'teamBID': teamBID,
                    'teamBBoats': teamBBoats,
                    'teamBScore': teamB['score'],
                    'teamBOutcome': teamB['outcome'],
                    })
    

df_final = pd.DataFrame(data)

(1/452) analyzing f16/hap-moore-team-race-backup
(2/452) analyzing f16/franklin-lane-backup
(3/452) analyzing f16/streufert
No score for this race? 16
No score for this race? 17
No score for this race? 18
No score for this race? 19
No score for this race? 20
No score for this race? 21
No score for this race? 22
No score for this race? 23
No score for this race? 24
No score for this race? 25
No score for this race? 26
No score for this race? 27
No score for this race? 28
No score for this race? 29
No score for this race? 30
No rp ented for this regatta f16/streufert
(4/452) analyzing f16/2016-laserperformance-collegiate-cup
No rp ented for this regatta f16/2016-laserperformance-collegiate-cup
(5/452) analyzing s16/laserperformance-team-race-nationals
No rp ented for this regatta s16/laserperformance-team-race-nationals
(6/452) analyzing s16/nwicsa-team-race
No rp ented for this regatta s16/nwicsa-team-race
(7/452) analyzing s16/seisa-team-racing-champs
No rp ented for this regatta s16/s

In [390]:
df_totalSailors2 = pd.DataFrame.from_dict(totalSailors,orient='index')
df_totalSailors2 = df_totalSailors2.reset_index(drop=True)
df_totalSailors2.to_json("trSailorInfoAll.json", index=False)
df_sailors

""


Brainstorming on datastructure: 
```
# By sailor
#columns = ['raceNum', 'round', 'name','link', 'pos', 'partner', 'teamName', 'fullTeamName', 'teamNick','teamLink', 'teamScore', 'win?', 'opponentTeamName','opponentFullName', 'opponentNick','opponentLink', 'opponentScore']

# By race
# potentially better because we need it by race for the elo calc anyways, and it will always be displayed by race in the regatta page. 
# downside is that it makes it harder to display on an individual sailor page, but the elo calc can handle that
# columns = ['raceNum', 'round',
#            'winTeamName', 'winTeamUni', 'winTeamNick', 'winTeamLink', 'winTeamID', 'winTeamBoatA', 'winTeamBoatB', 'winTeamBoatC','winTeamScore'
#            'loseTeamName', 'loseTeamUni', 'loseTeamNick', 'loseTeamLink', 'loseTeamID', 'loseTeamBoatA', 'loseTeamBoatB', 'loseTeamBoatC','loseTeamScore']
# boat = ['skipperName', 'skipperLink', 'crewName', 'crewLink']```

In [393]:
df_final.to_json("allseasonsteamraces.json", index=False)